# EMI-GRU Implementation for Peliculas Dataset

This notebook is an extension of the code available in https://github.com/microsoft/EdgeML

In [ ]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val

def bagData(X, Y, subinstanceLen, subinstanceStride):
    '''
    Takes x and y of shape
    [-1, 128, 9] and [-1, 6] respectively and converts it into bags of instances.
    returns [-1, numInstance, ]
    '''
    numClass = 2
    numSteps = 24 # Window length
    numFeats = 540 # Number of features
    assert X.ndim == 3
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0 # subinstance length = Number of readings for which the class signature occurs
    assert subinstanceStride <= numSteps  
    assert subinstanceStride >= 0 
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))

In [ ]:
subinstanceLen = 12
subinstanceStride = 3
extractedDir = '<enter extraction path> '
from os import mkdir
# WHEN YOU CHANGE THE ABOVE - CREATE A FOLDER 
# mkdir('64_16')  
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + '/%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

In [ ]:
from rnn import EMI_DataPipeline
from rnn import EMI_BasicLSTM, EMI_FastGRNN, EMI_FastRNN, EMI_GRU
from emirnnTrainer import EMI_Trainer, EMI_Driver
import utils
import os 
import numpy as np
import tensorflow as tf

In [ ]:
def gru_experiment_generator(params, path = './DSAAR/64_16/'):
    """
        Function that will generate the experiments to be run.
        Inputs : 
        (1) Dictionary params, to set the network parameters.
        (2) Name of the Model to be run from [EMI-LSTM, EMI-FastGRNN, EMI-GRU]
        (3) Path to the dataset, where the csv files are present.
    """
    
    #Copy the params into the gru_dict.
    gru_dict = {**params}
    
    #---------------------------PARAM SETTING----------------------#
    
    # Network parameters for our LSTM + FC Layer
    NUM_HIDDEN = params["NUM_HIDDEN"]
    NUM_TIMESTEPS = params["NUM_TIMESTEPS"]
    ORIGINAL_NUM_TIMESTEPS = params["ORIGINAL_NUM_TIMESTEPS"]
    NUM_FEATS = params["NUM_FEATS"]
    FORGET_BIAS = params["FORGET_BIAS"]
    NUM_OUTPUT = params["NUM_OUTPUT"]
    USE_DROPOUT = True if (params["USE_DROPOUT"] == 1) else False
    KEEP_PROB = params["KEEP_PROB"]

    # For dataset API
    PREFETCH_NUM = params["PREFETCH_NUM"]
    BATCH_SIZE = params["BATCH_SIZE"]

    # Number of epochs in *one iteration*
    NUM_EPOCHS = params["NUM_EPOCHS"]
    # Number of iterations in *one round*. After each iteration,
    # the model is dumped to disk. At the end of the current
    # round, the best model among all the dumped models in the
    # current round is picked up..
    NUM_ITER = params["NUM_ITER"]
    # A round consists of multiple training iterations and a belief
    # update step using the best model from all of these iterations
    NUM_ROUNDS = params["NUM_ROUNDS"]
    LEARNING_RATE = params["LEARNING_RATE"]

    # A staging direcory to store models
    MODEL_PREFIX = params["MODEL_PREFIX"]
    
    #----------------------END OF PARAM SETTING----------------------#
    
    #----------------------DATA LOADING------------------------------#
    
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')
    
    # BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
    # step of EMI/MI RNN
    BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
    BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
    BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
    NUM_SUBINSTANCE = x_train.shape[1]
    print("x_train shape is:", x_train.shape)
    print("y_train shape is:", y_train.shape)
    print("x_test shape is:", x_val.shape)
    print("y_test shape is:", y_val.shape)
    
    #----------------------END OF DATA LOADING------------------------------#    
    
    #----------------------COMPUTATION GRAPH--------------------------------#
    
    # Define the linear secondary classifier
    def createExtendedGraph(self, baseOutput, *args, **kwargs):
        W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
        B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
        y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
        self.output = y_cap
        self.graphCreated = True

    def restoreExtendedGraph(self, graph, *args, **kwargs):
        y_cap = graph.get_tensor_by_name('y_cap_tata:0')
        self.output = y_cap
        self.graphCreated = True

    def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
        if inference is False:
            feedDict = {self._emiGraph.keep_prob: keep_prob}
        else:
            feedDict = {self._emiGraph.keep_prob: 1.0}
        return feedDict

    EMI_GRU._createExtendedGraph = createExtendedGraph
    EMI_GRU._restoreExtendedGraph = restoreExtendedGraph

    if USE_DROPOUT is True:
        EMI_Driver.feedDictFunc = feedDictFunc
    
    inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
    emiGRU = EMI_GRU(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                            useDropout=USE_DROPOUT)
    emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy',
                             stepSize=LEARNING_RATE)

    tf.reset_default_graph()
    g1 = tf.Graph()    
    with g1.as_default():
        # Obtain the iterators to each batch of the data
        x_batch, y_batch = inputPipeline()
        # Create the forward computation graph based on the iterators
        y_cap = emiGRU(x_batch)
        # Create loss graphs and training routines
        emiTrainer(y_cap, y_batch)
        
    #------------------------------END OF COMPUTATION GRAPH------------------------------#
    
    #-------------------------------------EMI DRIVER-------------------------------------#
        
    with g1.as_default():
        emiDriver = EMI_Driver(inputPipeline, emiGRU, emiTrainer)

    emiDriver.initializeSession(g1)
    y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                          y_train=y_train, bag_train=BAG_TRAIN,
                                          x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                          numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                          numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                          numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                          fracEMI=0.5, updatePolicy='top-k', k=1)

    #-------------------------------END OF EMI DRIVER-------------------------------------#
    
    #-----------------------------------EARLY SAVINGS-------------------------------------#
    
    """
        Early Prediction Policy: We make an early prediction based on the predicted classes
        probability. If the predicted class probability > minProb at some step, we make
        a prediction at that step.
    """
    def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
        assert instanceOut.ndim == 2
        classes = np.argmax(instanceOut, axis=1)
        prob = np.max(instanceOut, axis=1)
        index = np.where(prob >= minProb)[0]
        if len(index) == 0:
            assert (len(instanceOut) - 1) == (len(classes) - 1)
            return classes[-1], len(instanceOut) - 1
        index = index[0]
        return classes[index], index

    def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
        predictionStep = predictionStep + 1
        predictionStep = np.reshape(predictionStep, -1)
        totalSteps = np.sum(predictionStep)
        maxSteps = len(predictionStep) * numTimeSteps
        savings = 1.0 - (totalSteps / maxSteps)
        if returnTotal:
            return savings, totalSteps
        return savings
    
    #--------------------------------END OF EARLY SAVINGS---------------------------------#
    
    #----------------------------------------BEST MODEL-----------------------------------#
    
    k = 2
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
    mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
    emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    total_savings = mi_savings + (1 - mi_savings) * emi_savings
    print('Savings due to MI-RNN : %f' % mi_savings)
    print('Savings due to Early prediction: %f' % emi_savings)
    print('Total Savings: %f' % (total_savings))
    
    # A slightly more detailed analysis method is provided. 
    df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
    #print (tabulate(df, headers=list(df.columns), tablefmt='grid'))
    
    gru_dict["k"] = k
    gru_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
    gru_dict["total_savings"] = total_savings
    gru_dict["detailed analysis"] = df
    gru_dict["y_test"] = BAG_TEST
    gru_dict["y_pred"] = bagPredictions
    
    #----------------------------------END OF BEST MODEL-----------------------------------#
    
    #----------------------------------PICKING THE BEST MODEL------------------------------#
    
    devnull = open(os.devnull, 'r')
    for val in modelStats:
        round_, acc, modelPrefix, globalStep = val
        emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
        predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)

        bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
        print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
        print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
        mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
        emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
        total_savings = mi_savings + (1 - mi_savings) * emi_savings
        print("Total Savings: %f" % total_savings)
        
    #-------------------------------END OF PICKING THE BEST MODEL--------------------------#

    return gru_dict

In [ ]:
dataset = 'Peliculas'
path = '<enter feature set path>'

#Choose model from among [lstm, fastgrnn, gru]
model = 'gru'

# Dictionary to set the parameters.
gru_params = {
    "NUM_HIDDEN" : 50,
    "NUM_TIMESTEPS" : 12,
    "ORIGINAL_NUM_TIMESTEPS" : 24,
    "NUM_FEATS" : 540,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 2,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 10,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

#Preprocess data, and load the train,test and validation splits.
gru_dict = gru_experiment_generator(gru_params, path)

#Create the directory to store the results of this run.

dirname = ""
dirname = "./Results" + ''.join(dirname) + "/"+dataset+"/"+model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(gru_dict,open(dirname + "/gru_dict_" + filename + ".pkl",mode='wb'))